In [6]:
with open('data/airbus_report.xhtml') as f:
    report = f.read()

report[:1000]

'<?xml version="1.0" encoding="UTF-8"?><html xmlns="http://www.w3.org/1999/xhtml" xmlns:link="http://www.xbrl.org/2003/linkbase" xmlns:xbrli="http://www.xbrl.org/2003/instance" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xbrldi="http://xbrl.org/2006/xbrldi" xmlns:iso4217="http://www.xbrl.org/2003/iso4217" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:utr="http://www.xbrl.org/2009/utr" xmlns:num="http://www.xbrl.org/dtr/type/numeric" xmlns:nonnum="http://www.xbrl.org/dtr/type/non-numeric" xmlns:xbrldt="http://xbrl.org/2005/xbrldt" xmlns:ref="http://www.xbrl.org/2006/ref" xmlns:ix="http://www.xbrl.org/2013/inlineXBRL" xmlns:ixt="http://www.xbrl.org/inlineXBRL/transformation/2020-02-12" xmlns:airbus="http://www.airbus.com" xml:lang="en" xmlns:esef_cor="https://www.esma.europa.eu/taxonomy/2022-03-24/esef_cor" xmlns:ifrs-full="https://xbrl.ifrs.org/taxonomy/2022-03-24/ifrs-full"><head><title></title><style type="text/css">#text

In [51]:
from html.parser import HTMLParser
from collections import Counter, defaultdict

class ReportParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.tags = defaultdict(list)
        self.xbrlTags = []
        
    def handle_starttag(self, tag, attr):
        if tag.startswith('ix') or tag.startswith('xbrl'):
            self.tags[tag].append(attr)
            if tag.startswith('ix'):
                xbrlTag = next((v for k, v in attr if k == 'name'), None)
                self.xbrlTags.append(xbrlTag)
            
    def handle_endtag(self, tag):
        pass
        
    def handle_datat(self, data):
        pass

parser = ReportParser()
parser.feed(report)
# for k, v in parser.tags.items():
#     print(f'{k}: {len(v)}')
#     print(f'{v[0]}\n')
Counter(parser.xbrlTags)

Counter({None: 478,
         'ifrs-full:Equity': 32,
         'ifrs-full:ProfitLoss': 24,
         'ifrs-full:OtherComprehensiveIncome': 22,
         'ifrs-full:ComprehensiveIncome': 22,
         'ifrs-full:IssueOfEquity': 20,
         'ifrs-full:IncreaseDecreaseThroughSharebasedPaymentTransactions': 20,
         'ifrs-full:DividendsPaid': 20,
         'ifrs-full:IncreaseDecreaseThroughChangeInEquityOfSubsidiaries': 20,
         'ifrs-full:IncreaseDecreaseThroughTreasuryShareTransactions': 20,
         'ifrs-full:ProfitLossAttributableToOwnersOfParent': 4,
         'ifrs-full:ProfitLossAttributableToNoncontrollingInterests': 4,
         'ifrs-full:CashAndCashEquivalents': 4,
         'ifrs-full:CashAndCashEquivalentsIfDifferentFromStatementOfFinancialPosition': 4,
         'ifrs-full:Revenue': 2,
         'ifrs-full:CostOfSales': 2,
         'ifrs-full:GrossProfit': 2,
         'ifrs-full:SellingExpense': 2,
         'ifrs-full:AdministrativeExpense': 2,
         'ifrs-full:ResearchAnd

In [36]:
parser.tags['xbrli:scenario'][0]

[]